# 01 Load modules

In [1]:
import pandas as pd 
import numpy as np
import json
import os
import re

In [2]:
base_df=pd.read_csv('/home/e077926/buscode_2023/05_E2_tata/data/receipt_raw_df.csv')
base_df = base_df.drop(columns=['Unnamed: 0'])
base_df.head()

,text_lines,doc_id,line_id
0,tan woon yann,X00016469612,1
1,book tak taman daya sdn bhd,X00016469612,2
2,b94 7w,X00016469612,3
3,no5 5557 59 jalan sagu 18,X00016469612,4
4,taman daya,X00016469612,5


# 02 create entities df

In [3]:
# Utiliser str.split pour diviser chaque ligne en mots
base_df['text_lines'] = base_df['text_lines'].str.split()
# Utiliser explode pour créer une ligne pour chaque mot tout en conservant l'ID
id_df =base_df.explode('text_lines', ignore_index=True)


In [4]:
id_df['text_lines']=id_df['text_lines'].astype(str)
id_df.head()

,text_lines,doc_id,line_id
0,tan,X00016469612,1
1,woon,X00016469612,1
2,yann,X00016469612,1
3,book,X00016469612,2
4,tak,X00016469612,2


# 03 Create label_df

In [5]:
entity_folder_path = "/home/e077926/buscode_2023/05_E2_tata/data/SROIE2019/train/entities/"

# Créer un DataFrame vide
label_df = pd.DataFrame(columns=['value', 'entity', 'doc'])
temp_df = pd.DataFrame(columns=['value', 'entity', 'doc'])
# Parcourir les fichiers dans le dossier
for entity_file in os.listdir(entity_folder_path):
    file_name, _ = os.path.splitext(entity_file)
    file_path = os.path.join(entity_folder_path, entity_file)

    # Charger le fichier JSON
    with open(file_path, 'r') as f:
        data_dict = json.load(f)

        # Parcourir les clés et valeurs du dictionnaire
        for k, v in data_dict.items():
            # Ajouter les clés et valeurs au DataFrame
            temp_df['value']=v, 
            temp_df['entity']=k
            temp_df['doc']=file_name

            
            label_df=pd.concat([label_df, temp_df], ignore_index=True)

# Afficher le DataFrame final
label_df['value'] = label_df['value'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
label_df['value'] = label_df['value'].str.lower()
label_df['value'] = label_df['value'].astype(str)
label_df.head()

,value,entity,doc
0,book ta k taman daya sdn bhd,company,X00016469612
1,25122018,date,X00016469612
2,no53 5557 59 jalan sagu 18 taman daya 81100 j...,address,X00016469612
3,900,total,X00016469612
4,indah gift home deco,company,X00016469619


In [6]:
label_df['value'] = label_df['value'].str.split()
# Utiliser explode pour créer une ligne pour chaque mot tout en conservant l'ID
label_df =label_df.explode('value', ignore_index=True)
label_df.head()

,value,entity,doc
0,book,company,X00016469612
1,ta,company,X00016469612
2,k,company,X00016469612
3,taman,company,X00016469612
4,daya,company,X00016469612


# 04 add entities labels

We'll use BIO (Begin, Inside, Outside)

'B-' (Begin) pour le premier mot d'une entité, 

'I-' (Inside) pour les mots suivants de la même entité

'O' (Outside) pour les mots qui ne font pas partie d'une entité. 

clean label_df

## 01 Add general entities

In [7]:
id_df['value']=id_df['text_lines']
id_df['doc']=id_df['doc_id']
id_df=id_df.drop(columns=['text_lines', 'doc_id'])
id_df.head()

,line_id,value,doc
0,1,tan,X00016469612
1,1,woon,X00016469612
2,1,yann,X00016469612
3,2,book,X00016469612
4,2,tak,X00016469612


In [8]:
new_df = pd.merge(id_df, label_df,  how='left', on = ['doc','value'])
new_df = new_df.reindex(columns=["doc", "line_id", "value", "entity"])
new_df.head()

,doc,line_id,value,entity
0,X00016469612,1,tan,NaN
1,X00016469612,1,woon,NaN
2,X00016469612,1,yann,NaN
3,X00016469612,2,book,company
4,X00016469612,2,tak,NaN


In [9]:
grouped_counts= new_df.groupby(['doc', 'line_id', 'entity']).size().reset_index(name='count')
grouped_counts['max'] = grouped_counts.groupby(['doc', 'line_id'])['count'].transform('max')
grouped_counts.head()

,doc,line_id,entity,count,max
0,X00016469612,2,address,2,5
1,X00016469612,2,company,5,5
2,X00016469612,4,address,5,5
3,X00016469612,5,address,2,2
4,X00016469612,5,company,2,2


In [10]:
# drop les valeurs qui ne sont pas égales au maxet donc en double du fait du merge
temp_df=new_df.merge(grouped_counts, how='left', on=['doc', 'line_id', 'entity'])
temp_df.drop(temp_df[temp_df['count']< temp_df['max']].index, inplace = True)
temp_df.head()

,doc,line_id,value,entity,count,max
0,X00016469612,1,tan,NaN,NaN,NaN
1,X00016469612,1,woon,NaN,NaN,NaN
2,X00016469612,1,yann,NaN,NaN,NaN
3,X00016469612,2,book,company,5.0,5.0
4,X00016469612,2,tak,NaN,NaN,NaN


In [11]:
# Créer une colonne indiquant si chaque ligne est un doublon
temp_df['is_duplicate'] = temp_df.duplicated(subset=['doc', 'line_id', 'value'], keep=False)

# Filtrer pour garder uniquement les lignes qui sont des doublons
duplicates = temp_df[temp_df['is_duplicate']]

# Filtrer les doublons pour garder seulement ceux dont 'entity' n'apparaît pas déjà dans le même 'doc'
filtered_duplicates = duplicates[~duplicates['entity'].isin(duplicates.groupby(['doc', 'value'])['entity'].transform('first'))]

# Concaténer les lignes à garder avec les lignes qui ne sont pas des doublons
result_df = pd.concat([temp_df[~temp_df['is_duplicate']], filtered_duplicates])

# Supprimer la colonne temporaire
result_df = result_df.drop(columns=['is_duplicate'])

# Afficher le DataFrame résultant
result_df.head()

,doc,line_id,value,entity,count,max
0,X00016469612,1,tan,NaN,NaN,NaN
1,X00016469612,1,woon,NaN,NaN,NaN
2,X00016469612,1,yann,NaN,NaN,NaN
3,X00016469612,2,book,company,5.0,5.0
4,X00016469612,2,tak,NaN,NaN,NaN


## 02 Add BIO

In [12]:
O_df=result_df.copy()
O_df['entity'] = O_df['entity'].fillna('O')
# Créer une nouvelle colonne pour les entités avec les suffixes
BIO_df=O_df.copy().reset_index()
BIO_df=BIO_df.drop(columns=['count','max', 'index'])
BIO_df.head()

,doc,line_id,value,entity
0,X00016469612,1,tan,O
1,X00016469612,1,woon,O
2,X00016469612,1,yann,O
3,X00016469612,2,book,company
4,X00016469612,2,tak,O


In [13]:
# Add -B and -I suffixes
for i in range(len(BIO_df)):
    entity = BIO_df.at[i, 'entity']

    if entity != 'O':
        # Si c'est la première occurrence, ajouter '-B', sinon '-I'
        suffix = '-B' if BIO_df.at[i-1, 'entity'] == 'O' else '-I'
        BIO_df.at[i, 'entity'] = entity + suffix  

In [15]:
BIO_df.to_csv('/home/e077926/buscode_2023/05_E2_tata/data/bio_df.csv')